In [ ]:
import time
import random
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from PIL import Image
import requests
import os

In [ ]:
option = webdriver.ChromeOptions()
option.add_argument("-incognito")
option.add_argument("--headless")
option.add_argument("disable-gpu");
# Replace the path in the following line with the path for your chromedrive
browser = webdriver.Chrome(executable_path=r'/home/srujandeshpande/chromedriver', options=option)

In [ ]:
subscription_key = os.environ['OCR_SUBSCRIPTION_KEY']
endpoint = os.environ['OCR_ENDPOINT']

In [ ]:
# Replace the URL with whichever website you are targeting. Further changes need to be made at certain places to account for different websites.
# UPDATE: This code was originally written for Vahan which has since migrated to a different captcha system. The current code is unlikely to work and is untested on the updated site.
browser.get("https://vahan.nic.in/nrservices/faces/user/searchstatus.xhtml")

In [ ]:
browser.save_screenshot("image.png") 

In [ ]:
# Replace with XPATH of the captcha
e = browser.find_element_by_xpath('//*[@id="capatcha"]/img')

location = dict(e.location)
size = dict(e.size)
print(location,size)

In [ ]:
im = Image.open("image.png") 
  
# Setting the points for cropped image 
left = location['x']
top = location['y']-10
right = location['x']+size['width']
bottom = location['y']+size['height']+10
  
# Cropped image of above dimensions 
im1 = im.crop((left, top, right, bottom)) 
im1.save("cropped.png")

In [ ]:
ocr_url = endpoint + "vision/v3.0/read/analyze"

image_data = open("cropped.png", "rb").read()

headers = {'Ocp-Apim-Subscription-Key': subscription_key, 'Content-Type': 'application/octet-stream'}
params = {'language': 'en'}

response = requests.post(ocr_url, headers=headers, params=params, data=image_data)

response.raise_for_status()

In [ ]:
result = dict()

analysis = {}
poll = True
while (poll):
    response_final = requests.get(
        response.headers["Operation-Location"], headers=headers)
    analysis = response_final.json()
    
    result = json.dumps(analysis, indent=2)

    time.sleep(1)
    if ("analyzeResult" in analysis):
        poll = False
    if ("status" in analysis and analysis['status'] == 'failed'):
        poll = False

print(result)

In [ ]:
text = analysis['analyzeResult']['readResults'][0]['lines'][0]['text']

words = text.split()

answer = ""

if words[0]=='Greater':
    num1 = words[2][:-1]
    num2 = words[3]
    
    answer = num1 if num1 > num2 else num2
elif words[0]=='Smallest':
    num1 = words[2][:-1]
    num2 = words[3]
    
    answer = num1 if num1 < num2 else num2
elif words[1]=='+':
    num1 = int(words[0])
    num2 = int(words[2])

    answer = num1 + num2  
elif words[1]=='-':
    num1 = int(words[0])
    num2 = int(words[2])

    answer = num1 - num2  
elif words[1]=='*':
    num1 = int(words[0])
    num2 = int(words[2])

    answer = num1 * num2  
elif words[1]=='/':
    num1 = int(words[0])
    num2 = int(words[2])

    answer = num1 / num2  
answer = str(answer)
print(f"{text}: {answer}")

In [ ]:
vehicle_number = browser.find_element_by_xpath('//*[@id="regn_no1_exact"]')
answer_field = browser.find_element_by_xpath('//*[@id="txt_ALPHA_NUMERIC"]')
enter = browser.find_element_by_xpath('//*[@id="j_idt25"]/span')

vehicle_number.send_keys('CKN8941')
answer_field.send_keys(answer)
enter.click()